In [1]:
import numpy as np
import pandas as pd
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
from scipy.stats import shapiro

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:
df = pd.read_csv('alzheimers_disease_data.csv', sep = ',')

In [3]:
import pandas as pd
import statsmodels.api as sm

df['intercept'] = 1

logit_model = sm.Logit(df['Diagnosis'], df[['intercept', 'SystolicBP', 'DiastolicBP', 'CholesterolTotal', 'CholesterolLDL', 'CholesterolHDL', 'CholesterolTriglycerides', 'MMSE', 'FunctionalAssessment', 'MemoryComplaints', 'BehavioralProblems', 'ADL', 'Confusion', 'Disorientation', 'PersonalityChanges', 'DifficultyCompletingTasks', 'Forgetfulness']])
result = logit_model.fit()

print(result.summary())


Optimization terminated successfully.
         Current function value: 0.371584
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:              Diagnosis   No. Observations:                 2149
Model:                          Logit   Df Residuals:                     2132
Method:                           MLE   Df Model:                           16
Date:                Sun, 01 Sep 2024   Pseudo R-squ.:                  0.4280
Time:                        21:31:13   Log-Likelihood:                -798.53
converged:                       True   LL-Null:                       -1396.2
Covariance Type:            nonrobust   LLR p-value:                1.556e-244
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
intercept                     3.6424      0.670      5.438      0.000       2.

Статистическая значимость: Некоторые переменные статистически значимы (p-value < 0.05), например, MMSE, FunctionalAssessment, MemoryComplaints, BehavioralProblems, ADL. Другие переменные, такие как SystolicBP, DiastolicBP, CholesterolTotal, не являются статистически значимыми. Pseudo R-squared: Значение псевдо R-квадрата равно 0.4280. Это говорит о том, что модель объясняет 42.8% вариации в данных. Заключение: модель имеет неплохую объяснительную способность, но некоторые переменные не являются статистически значимыми.

Гипотеза: различные клинические измерения и показатели могут быть связаны с риском развития болезни (диагноза Alzheimer’s Disease).

Интерпретация результатов:

Некоторые переменные, такие как MMSE, FunctionalAssessment, MemoryComplaints, BehavioralProblems, ADL, статистически значимы и имеют влияние на вероятность болезни. Другие переменные, такие как SystolicBP, DiastolicBP, CholesterolTotal, не являются статистически значимыми.

 использование модели логистической регрессии для предсказания вероятности развития болезни Альцгеймера

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import classification_report, confusion_matrix

data = pd.read_csv('alzheimers_disease_data.csv', sep=',')


data.drop(columns=['DoctorInCharge', 'PatientID'], inplace=True)


X = data.drop(columns=['Diagnosis'])
y = data['Diagnosis']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

logreg = LogisticRegression(max_iter=1000)


logreg.fit(X_train_scaled, y_train)


rfe = RFE(logreg, n_features_to_select=10)  # Выберите желаемое количество признаков
rfe.fit(X_train_scaled, y_train)


selected_features = X.columns[rfe.support_]
print("Отобранные признаки:", selected_features)

Отобранные признаки: Index(['Age', 'EducationLevel', 'Smoking', 'DiastolicBP', 'CholesterolHDL',
       'MMSE', 'FunctionalAssessment', 'MemoryComplaints',
       'BehavioralProblems', 'ADL'],
      dtype='object')


In [32]:
scaler_selected = StandardScaler()
X_train_selected_scaled = scaler_selected.fit_transform(X_train[selected_features])
X_test_selected_scaled = scaler_selected.transform(X_test[selected_features])

In [33]:

logreg.fit(X_train_selected_scaled, y_train)


y_pred = logreg.predict(X_test_selected_scaled)

In [34]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.90      0.87       277
           1       0.80      0.71      0.75       153

    accuracy                           0.83       430
   macro avg       0.82      0.81      0.81       430
weighted avg       0.83      0.83      0.83       430



In [35]:
conf_matrix = confusion_matrix(y_test, y_pred)
print("Матрица ошибок:")
print(conf_matrix)

Матрица ошибок:
[[249  28]
 [ 44 109]]


Точность (Precision): Для класса 0 (отрицательный диагноз) точность составляет 85%, что означает, что из всех предсказанных отрицательных диагнозов 85% действительно являются отрицательными. Для класса 1 (положительный диагноз) точность составляет 80%, что означает, что из всех предсказанных положительных диагнозов 80% действительно являются положительными. Полнота (Recall): Для класса 0 полнота составляет 90%, что означает, что из всех действительных отрицательных диагнозов 90% были правильно предсказаны. Для класса 1 полнота составляет 71%, что означает, что из всех действительных положительных диагнозов 71% были правильно предсказаны. F1-мера (F1-score): F1-мера объединяет точность и полноту в одну метрику. Она равна среднему гармоническому между точностью и полнотой. Взвешенная F1-мера составляет 83%, что является хорошим показателем. Матрица ошибок: Верхний левый элемент (249) представляет количество верно предсказанных отрицательных диагнозов (True Negatives). Верхний правый элемент (28) представляет количество ложно предсказанных положительных диагнозов (False Positives). Нижний левый элемент (44) представляет количество ложно предсказанных отрицательных диагнозов (False Negatives). Нижний правый элемент (109) представляет количество верно предсказанных положительных диагнозов (True Positives). Общая точность модели составляет 83%, что говорит о том, что она хорошо справляется с прогнозированием диагноза.

In [36]:
patient_features = np.array([[65, 3, 0, 80, 50, 25, 1, 1, 0, 1]])  # Пример значений
patient_features_df = pd.DataFrame(patient_features, columns=selected_features)
patient_features_scaled = scaler_selected.transform(patient_features_df)

In [37]:
predicted_probability = logreg.predict_proba(patient_features_scaled)[0][1]
print(f"Вероятность развития болезни Альцгеймера: {predicted_probability:.2f}")

Вероятность развития болезни Альцгеймера: 0.95
